In [1]:
%%js
$(".repository-content, .container").css("width", "90%");
$("body, #outer-dashboard, .navbar-default").css("background-color", "papayawhip");
$(".navbar-default").css("background-color", "lightblue");

<IPython.core.display.Javascript object>

In [2]:
%%html
<img src="//localhost:3010/files/img/pydata.png">

In [3]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, SingleIntervalTicker, CategoricalColorMapper, Label
from bokeh.palettes import Spectral6
from bokeh.plotting import figure

from bokeh.sampledata.gapminder import fertility, life_expectancy, population, regions

In [4]:
from ipywidgets import widgets, interact
from IPython.display import display

In [5]:
def process_data():
    from bokeh.sampledata.gapminder import fertility, life_expectancy, population, regions

    # Make the column names ints not strings for handling
    columns = list(fertility.columns)
    years = list(range(int(columns[0]), int(columns[-1])))
    rename_dict = dict(zip(columns, years))

    fertility = fertility.rename(columns=rename_dict)
    life_expectancy = life_expectancy.rename(columns=rename_dict)
    population = population.rename(columns=rename_dict)
    regions = regions.rename(columns=rename_dict)

    regions_list = list(regions.Group.unique())

    # Turn population into bubble sizes. Use min_size and factor to tweak.
    scale_factor = 200
    population_size = pd.DataFrame(data=np.sqrt(population.values / np.pi) / scale_factor, columns=population.columns, index=population.index)
    min_size = 3
    population_size = population_size.where(
        population_size >= min_size).fillna(min_size)

    return fertility, life_expectancy, population_size, regions, years, regions_list


In [6]:
output_notebook(hide_banner=True)

In [7]:
fertility_df, life_expectancy_df, population_df_size, regions_df, years, regions_list = process_data()

p = pd.Panel({'fertility': fertility_df, 'life': life_expectancy_df,
              'population': population_df_size})

data = {}
region_name = regions_df.Group
region_name.name = 'region'

for year in years:
    df = pd.concat([p.loc[:, :, year], region_name], axis=1).reset_index()
    data[year] = df.to_dict('series')

source = ColumnDataSource(data=data[years[0]])

In [8]:
plot = figure(
    x_range=(1, 9), 
    y_range=(20, 100),
    title='Gapminder Reborn', 
    plot_height=300,
    sizing_mode='scale_width',
)

plot.background_fill_alpha = 0
plot.border_fill_alpha = 0

plot.title.text_font = 'Verdana'
plot.title.text_font_size = '30pt'
plot.title.text_color = 'steelblue'

plot.xaxis.axis_label = "Children per woman (total fertility)"
plot.yaxis.axis_label = "Life expectancy at birth (years)"

label = Label(x=1.1, y=18, text=str(years[0]), text_font_size='70pt', text_color='steelblue')
plot.add_layout(label)

color_mapper = CategoricalColorMapper(palette=Spectral6, factors=regions_list)
plot.circle(
    x='fertility',
    y='life',
    size='population',
    source=source,
    fill_color={'field': 'region', 'transform': color_mapper},
    fill_alpha=0.8,
    line_color='#7c7e71',
    line_width=0.5,
    line_alpha=0.5,
    legend='region',
    hover_fill_alpha=0.3,
    hover_fill_color={'field': 'region', 'transform': color_mapper}
 )
plot.add_tools(HoverTool(tooltips="@index", show_arrow=False, point_policy='follow_mouse'))

In [9]:
plot.legend.background_fill_color = "steelblue"
plot.legend.background_fill_alpha = 0.1

In [10]:
handle = show(plot, notebook_handle=True);

In [11]:
slider = widgets.IntSlider(min=years[0], max=years[-1], value=1980, height='300px', description='Year', orientation='vertical')
debug_widget = widgets.HTML('foo')
display(debug_widget)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [12]:
@interact(year=slider)
def update(year):
    source.data = data[year]
    label.text = str(year)
    push_notebook(handle=handle)
    debug_widget.value = '<h3>{} AD</h3>'.format(year)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
